In [1]:
import os
import numpy as np
from data_generator import DataGenerator

In [2]:
label_name = {}
for index, folder in enumerate(os.listdir(f'data/train')):
    label_name[folder] = index-1

In [3]:
train_paths = np.array([])
train_labels = {}
for folder in os.listdir('data/train'):
    for file in os.listdir(f'data/train/{folder}'):
        # required due to .DS_Store
        if file[0] == '.':
            continue
        train_paths = np.append(train_paths, np.array([f'data/train/{folder}/{file}']))
        train_labels[f'data/train/{folder}/{file}'] = label_name[folder]

validate_paths = np.array([])
validate_labels = {}
for folder in os.listdir('data/validation'):
    for file in os.listdir(f'data/validation/{folder}'):
        if file[0] == '.':
            continue
        validate_paths = np.append(validate_paths, np.array([f'data/validation/{folder}/{file}']))
        validate_labels[f'data/validation/{folder}/{file}'] = label_name[folder]

In [4]:
train_generator = DataGenerator(train_paths, train_labels, batch_size=128, n_channels=3)
validate_generator = DataGenerator(validate_paths, validate_labels, batch_size=128, n_channels=3)

In [7]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, Dense, GlobalMaxPooling2D

model = Sequential([
    Conv2D(filters=16, kernel_size=(3,3), input_shape=(250,250,3), activation='relu', padding='same'),
    Conv2D(filters=16, kernel_size=(2,2), strides=(2,2), name='Max_pooling_1'),
    Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same'),
    Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same'),
    Conv2D(filters=32, kernel_size=(2,2), strides=(2,2), name='Max_pooling_2'),
    Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
    Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
    Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
    GlobalMaxPooling2D(),
    Dense(5, activation='softmax')
])

In [8]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics='accuracy')

In [ ]:
os.makedirs('models')

In [6]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
callback = [ModelCheckpoint(f'models/2.h5', monitor='val_loss', verbose=1, save_best_only=True)]

In [5]:
from tensorflow.keras.models import load_model
model = load_model('models/2.h5')

In [37]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_47 (Conv2D)          (None, 250, 250, 16)      448       
                                                                 
 Max_pooling_1 (Conv2D)      (None, 125, 125, 16)      1040      
                                                                 
 conv2d_48 (Conv2D)          (None, 125, 125, 32)      4640      
                                                                 
 conv2d_49 (Conv2D)          (None, 125, 125, 32)      9248      
                                                                 
 Max_pooling_2 (Conv2D)      (None, 62, 62, 32)        4128      
                                                                 
 conv2d_50 (Conv2D)          (None, 62, 62, 64)        18496     
                                                                 
 conv2d_51 (Conv2D)          (None, 60, 60, 64)       

In [7]:
model.fit(train_generator, epochs=45, batch_size=128, validation_data=validate_generator, callbacks=callback)

Epoch 1/45
468/468 [==============================] - ETA: 0s - loss: 0.0103 - accuracy: 0.9966
Epoch 1: val_loss improved from inf to 0.02348, saving model to models\2.h5
468/468 [==============================] - 133s 275ms/step - loss: 0.0103 - accuracy: 0.9966 - val_loss: 0.0235 - val_accuracy: 0.9931
Epoch 2/45
468/468 [==============================] - ETA: 0s - loss: 0.0066 - accuracy: 0.9979
Epoch 2: val_loss improved from 0.02348 to 0.01579, saving model to models\2.h5
468/468 [==============================] - 130s 278ms/step - loss: 0.0066 - accuracy: 0.9979 - val_loss: 0.0158 - val_accuracy: 0.9962
Epoch 3/45
468/468 [==============================] - ETA: 0s - loss: 0.0131 - accuracy: 0.9957
Epoch 3: val_loss did not improve from 0.01579
468/468 [==============================] - 126s 270ms/step - loss: 0.0131 - accuracy: 0.9957 - val_loss: 0.0196 - val_accuracy: 0.9934
Epoch 4/45
468/468 [==============================] - ETA: 0s - loss: 0.0104 - accuracy: 0.9965
Epoch 4:

In [16]:
test_paths = np.array([])
test_labels = {}
for folder in os.listdir('data/test'):
    for file in os.listdir(f'data/test/{folder}'):
        if file[0] == '.':
            continue
        test_paths = np.append(test_paths, np.array([f'data/test/{folder}/{file}']))
        test_labels[f'data/test/{folder}/{file}'] = label_name[folder]

In [17]:
test_generator = DataGenerator(test_paths, test_labels, batch_size=128, n_channels=3)

In [18]:
model.evaluate(test_generator)

58/58 [==============================] - 14s 236ms/step - loss: 0.0140 - accuracy: 0.9987


[0.014046893455088139, 0.998652994632721]

In [ ]:
model.evaluate(test_generator)

Test performance differences when MaxPooling is used.

In [10]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, GlobalMaxPooling2D

model = Sequential([
    Conv2D(filters=16, kernel_size=(3,3), input_shape=(250,250,3), activation='relu', padding='same'),
    MaxPooling2D(),
    Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same'),
    Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same'),
    MaxPooling2D(),
    Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
    Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
    Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
    GlobalMaxPooling2D(),
    Dense(5, activation='softmax')
])

In [11]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics='accuracy')

In [12]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
callback = [ModelCheckpoint(f'models/2_max_pooling.h5', monitor='val_loss', verbose=1, save_best_only=True)]

In [13]:
model.fit(train_generator, epochs=100, batch_size=128, validation_data=validate_generator, callbacks=callback)

Epoch 1/100
468/468 [==============================] - ETA: 0s - loss: 0.5627 - accuracy: 0.7677
Epoch 1: val_loss improved from inf to 0.08581, saving model to models\2_max_pooling.h5
468/468 [==============================] - 132s 272ms/step - loss: 0.5627 - accuracy: 0.7677 - val_loss: 0.0858 - val_accuracy: 0.9721
Epoch 2/100
468/468 [==============================] - ETA: 0s - loss: 0.1077 - accuracy: 0.9640
Epoch 2: val_loss did not improve from 0.08581
468/468 [==============================] - 129s 275ms/step - loss: 0.1077 - accuracy: 0.9640 - val_loss: 0.1145 - val_accuracy: 0.9609
Epoch 3/100
468/468 [==============================] - ETA: 0s - loss: 0.0601 - accuracy: 0.9803
Epoch 3: val_loss improved from 0.08581 to 0.05191, saving model to models\2_max_pooling.h5
468/468 [==============================] - 128s 272ms/step - loss: 0.0601 - accuracy: 0.9803 - val_loss: 0.0519 - val_accuracy: 0.9824
Epoch 4/100
468/468 [==============================] - ETA: 0s - loss: 0.0615

In [14]:
model.evaluate(test_generator)

58/58 [==============================] - 15s 249ms/step - loss: 0.0187 - accuracy: 0.9976


[0.018660806119441986, 0.9975754022598267]